In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Esophageal_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Esophageal_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Esophageal_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Esophageal_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Esophageal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. Find the most relevant directory for Colon and Rectal Cancer
subdirectories = os.listdir(tcga_root_dir)
target_trait = trait.lower().replace("_", " ")  # Convert to lowercase for case-insensitive matching

# Start with no match, then find the best match based on similarity to target trait
best_match = None
best_match_score = 0

for subdir in subdirectories:
    subdir_lower = subdir.lower()
    
    # Calculate a simple similarity score - more matching words = better match
    # This prioritizes exact matches over partial matches
    score = 0
    for word in target_trait.split():
        if word in subdir_lower:
            score += 1
    
    # Track the best match
    if score > best_match_score:
        best_match_score = score
        best_match = subdir
        print(f"Found potential match: {subdir} (score: {score})")

# Use the best match if found
if best_match:
    print(f"Selected directory: {best_match}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, best_match)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not best_match:
    print("Skipping this trait as no suitable data was found.")


Found potential match: TCGA_Liver_Cancer_(LIHC) (score: 1)
Found potential match: TCGA_Esophageal_Cancer_(ESCA) (score: 2)
Selected directory: TCGA_Esophageal_Cancer_(ESCA)
Clinical file: TCGA.ESCA.sampleMap_ESCA_clinicalMatrix
Genetic file: TCGA.ESCA.sampleMap_HiSeqV2_PANCAN.gz



Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_treatment_completion_success_outcome', 'age_at_initial_pathologic_diagnosis', 'age_began_smoking_in_years', 'alcohol_history_documented', 'amount_of_alcohol_consumption_per_day', 'antireflux_treatment_type', 'barretts_esophagus', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'city_of_procurement', 'clinical_M', 'clinical_N', 'clinical_T', 'clinical_stage', 'columnar_metaplasia_present', 'columnar_mucosa_dysplasia', 'columnar_mucosa_goblet_cell_present', 'country_of_birth', 'country_of_procurement', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'eastern_cancer_oncology_group', 'esophageal_

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify candidate age and gender columns
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'age_began_smoking_in_years', 'days_to_birth']
candidate_gender_cols = ['gender']

# 2. Load the clinical data from previous step
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Esophageal_Cancer_(ESCA)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
clinical_data = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Preview age columns
age_preview = {col: clinical_data[col].head(5).tolist() for col in candidate_age_cols}
print("Age columns preview:")
print(age_preview)

# Preview gender columns
gender_preview = {col: clinical_data[col].head(5).tolist() for col in candidate_gender_cols}
print("Gender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [67, 66, 44, 68, 57], 'age_began_smoking_in_years': [nan, nan, nan, nan, nan], 'days_to_birth': [-24487, -24328, -16197, -25097, -21180]}
Gender columns preview:
{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine age columns
age_candidate_columns = {
    'age_at_initial_pathologic_diagnosis': [67, 66, 44, 68, 57],
    'age_began_smoking_in_years': [float('nan'), float('nan'), float('nan'), float('nan'), float('nan')],
    'days_to_birth': [-24487, -24328, -16197, -25097, -21180]
}

# Examine gender columns
gender_candidate_columns = {
    'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'MALE']
}

# Select the best column for age
# 'age_at_initial_pathologic_diagnosis' has all valid values
# 'days_to_birth' is negative days, could be converted but less intuitive
# 'age_began_smoking_in_years' has all NaN values
age_col = 'age_at_initial_pathologic_diagnosis'

# Select the best column for gender
# 'gender' is the only option and has valid values
gender_col = 'gender'

# Print the selected columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 196)
Clinical data saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/TCGA.csv
Clinical data shape: (204, 3)
Number of samples in clinical data: 204
Number of samples in genetic data: 196
Number of common samples: 196
Linked data shape: (196, 19851)


Data shape after handling missing values: (196, 19851)
For the feature 'Esophageal_Cancer', the least common label is '0' with 11 occurrences. This represents 5.61% of the dataset.
The distribution of the feature 'Esophageal_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 54.0
  50% (Median): 61.0
  75%: 72.0
Min: 27
Max: 90
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 31 occurrences. This represents 15.82% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Esophageal_Cancer/TCGA.csv
Preprocessing completed.
